## Create a Weaviate database


In [7]:
import weaviate
import os
  
# Set these environment variables
URL = "https://pillar-74yhe0v2.weaviate.network"
APIKEY = "l6eCVSOBiI6eps5KVzLoyO7nnvBpCzQNCkCb"
  
# Connect to a WCS instance
client = weaviate.connect_to_wcs(
    skip_init_checks=True, # https://forum.weaviate.io/t/grpc-is-not-available-please-make-sure-that-grpc-is-configured-correctly-in-the-client-and-on-the-server-weaviate-v1-23-10/1634/4?u=pillar_liang
    cluster_url=URL,
    auth_credentials=weaviate.auth.AuthApiKey(APIKEY),
    )

client.is_ready()

True

## collect dataset


#### load document


In [67]:
from langchain.document_loaders import PyPDFLoader
    
loader = PyPDFLoader(f"../高速公路养护管理办法.pdf")
contents = ""
pages = loader.load()
for page in pages:
    contents += page.page_content
    

#### split document


##### method1


In [68]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
# chunks  = text_splitter.split_documents(pages) # 这样划分，无论chunk_size是多少，chunks 都是按页划分的。
chunks = text_splitter.split_text(contents)

print("chunks:", len(chunks))

chunks: 8


In [63]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

vectorstore = Weaviate.from_texts(
    client = client,
    texts = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

{"level":"info","msg":"Created shard langchain_43546acc30594bb98a658b698b31c99a_5K3Kn9tJVjHb in 2.644043ms","time":"2024-04-07T16:18:33+08:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-04-07T16:18:33+08:00","took":77787}
/Users/liangzhu/anaconda3/envs/llm/lib/python3.12/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


##### method2


In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
chunks  = text_splitter.split_documents(pages) # 这样划分，无论chunk_size是多少，chunks 都是按页划分的。

print("chunks:", len(chunks))

chunks: 9


In [66]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

{"level":"info","msg":"Created shard langchain_d86e03356e824cc081ac8c955e89f612_7CS6MtH475Q7 in 2.902596ms","time":"2024-04-07T16:20:13+08:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-04-07T16:20:13+08:00","took":70003}
/Users/liangzhu/anaconda3/envs/llm/lib/python3.12/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


## RAG


#### 1. retriever


In [31]:
retriever = vectorstore.as_retriever()

#### 2. Augment


In [32]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))]


#### 3. Generate


In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "养护分公司的职责是什么？"
rag_chain.invoke(query)

/Users/liangzhu/anaconda3/envs/llm/lib/python3.12/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/liangzhu/anaconda3/envs/llm/lib/python3.12/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'养护分公司的职责包括审核计量常规养护费用和养护工程费用，统一管理路产索赔资金，组织技术状况检测评定，负责养护技术研究与开发，推广新材料、新技术、新工艺、新设备在养护工程中的应用，以及指导养护业务和统计分析信息资料。其职能主要包括贯彻执行相关法律法规，组织实施管辖路段养护生产、路产保护、道路救援等工作，承担内业档案和技术管理职责，组织、检查、指导养护工区开展日常维保、除雪、绿化、小修工程等工作。养护分公司还负责调度指挥应急救援的资源调配、清障救援、抢险救灾、公路及公路设施的抢通和修复等工作。'